In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
from larndsim import consts, quenching, drifting, detsim, pixels_from_track
import importlib
importlib.reload(drifting)
importlib.reload(quenching)
importlib.reload(pixels_from_track)
importlib.reload(consts)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import pickle
import numpy as np
import numba as nb
import pandas as pd
import time

from numba import cuda
import cupy as cp
from math import ceil

In [3]:
tracks_dict = pickle.load(open('edepsimDataset.p', 'rb'))
tracks = pd.DataFrame.from_dict(tracks_dict)
tracks['t'] = 0.
tracks['t_start'] = 0.
tracks['t_end'] = 0.
tracks['x_start'] = tracks_dict['z_start'] 
tracks['z_start'] = tracks_dict['x_start'] 
tracks['x_end'] = tracks_dict['z_end'] 
tracks['z_end'] = tracks_dict['x_end'] 

tracks['z_start'] /= 1e2
tracks['z_end'] /= 1e2
tracks['y_start'] /= 1e2
tracks['y_end'] /= 1e2
tracks['x_start'] /= 1e2
tracks['x_end'] /= 1e2

tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) +
                    pow(tracks['y_end']-tracks['y_start'], 2) +
                    pow(tracks['z_end']-tracks['z_start'], 2))
tracks['x'] = (tracks['x_end']+tracks['x_start'])/2
tracks['y'] = (tracks['y_end']+tracks['y_start'])/2
tracks['z'] = (tracks['z_end']+tracks['z_start'])/2
tracks['dE'] = np.abs(tracks['dE'])*1e3
tracks['dEdx'] = tracks['dE']/tracks['dx']
tracks['NElectrons'] = 0.
tracks['longDiff'] = 0.
tracks['tranDiff'] = 0.

tracks_np = tracks.to_numpy()
cols = nb.typed.Dict()
for i, c, in enumerate(tracks.columns):
    cols[c] = i

In [4]:
quenching.Quench(tracks_np, cols, mode="box")
drifting.Drift(tracks_np, cols)

In [5]:
x_start_device = cuda.to_device(tracks['x_start'])
x_end_device = cuda.to_device(tracks['x_end'])
y_start_device = cuda.to_device(tracks['y_start'])
y_end_device = cuda.to_device(tracks['y_end'])

In [6]:
active_pixels_device = cp.zeros(shape =(x_start_device.shape[0], 10, 2))
neighboring_pixels_device = cp.zeros(shape =(x_start_device.shape[0], 100, 2))
n_pixels_device = cp.zeros(shape =(x_start_device.shape[0]))

In [7]:
TPB = 128
BPG = ceil(x_start_device.shape[0] / TPB)

In [8]:
pixels_from_track.get_pixels[TPB,BPG](x_start_device, y_start_device, x_end_device, y_end_device, 
                                      active_pixels_device, neighboring_pixels_device, n_pixels_device)

In [9]:
n = cp.argmax(n_pixels_device)
n_pixels_device[n]

array(66.)

In [10]:
active_pixels_device[n]

array([[188., 168.],
       [189., 169.],
       [190., 170.],
       [190., 171.],
       [191., 172.],
       [192., 173.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.]])

In [11]:
neighboring_pixels_device[n]

array([[186., 166.],
       [186., 167.],
       [186., 168.],
       [186., 169.],
       [186., 170.],
       [187., 166.],
       [187., 167.],
       [187., 168.],
       [187., 169.],
       [187., 170.],
       [188., 166.],
       [188., 167.],
       [188., 168.],
       [188., 169.],
       [188., 170.],
       [189., 166.],
       [189., 167.],
       [189., 168.],
       [189., 169.],
       [189., 170.],
       [190., 166.],
       [190., 167.],
       [190., 168.],
       [190., 169.],
       [190., 170.],
       [187., 171.],
       [188., 171.],
       [189., 171.],
       [190., 171.],
       [191., 167.],
       [191., 168.],
       [191., 169.],
       [191., 170.],
       [191., 171.],
       [188., 172.],
       [189., 172.],
       [190., 172.],
       [191., 172.],
       [192., 168.],
       [192., 169.],
       [192., 170.],
       [192., 171.],
       [192., 172.],
       [188., 173.],
       [189., 173.],
       [190., 173.],
       [191., 173.],
       [192.,

In [12]:
total_pixels = cp.sum(n_pixels_device)
pixels = cp.zeros(shape=(int(total_pixels)))
trackID = cp.zeros_like(pixels)
pixelID = cp.zeros(shape = (int(total_pixels),2))